# Financial Networks and Contagion

"Financial Networks and Contagion", Matthew Elliot, Benjamin Golub and Matthew O. Jackson, *American Economic Review*, 2014

In [1]:
import numpy as np
import copy

## Model

- $N$ : the set of organizations
- $M$ : the set of assets
- $D$ : the matrix whose entry is $D_{ik}$, which is a share of asset $k$ held by organization $i$
- $C$ : the matrix whose entry is $C_{ij}$, which is a fraction of organization $j$ owned by organization $i$, but the diagonal elements are 0
- $\hat{C}$ : the diagonal matrix whose entry is $\hat{C}_{ii} := 1 - \sum_{j \in N} C_{ji}$, that is, a share of organization $i$ not owned by any organizations 
- $P$ : the vector of asset prices

The vector of organizations' book values : $V$
\begin{equation}
V = {(I - C)}^{-1} Dp
\end{equation}

$A$ : dependency matrix
\begin{equation}
\hat{C} {(I - C)}^{-1}
\end{equation}

The vector of organizations' values : $v$
\begin{equation}
v = \hat{C} V = \hat{C} {(I - C)}^{-1} Dp = ADp
\end{equation}

If the value $v_i$ falls below some threshhold level $\underline{v}_i$, then $i$ is said to fail and incurs the costs $\beta_i(P)$.

## Algorithm

At step $t$ of  the algorithm, let $\mathcal{Z}_t$ be the set of failed organizations.

Initialize $\mathcal{Z}_0 = \emptyset$. At step $t \ge 1$:

(1) Let $\tilde{b}_{t-1}$ be a vector with element $\tilde{b}_i = \beta_i$ if $i \in \mathcal{Z}_{t-1}$ and $0$ otherwise.

(2) Let $\mathcal{Z}_t$ be the set of all $k$ such that entry $k$ of the following vector is negative:

\begin{equation}
A[Dp - \tilde{b}_{t-1}] - \underline{v}
\end{equation}

(3) Terminate if $\mathcal{Z}_t = \mathcal{Z}_{t-1}$. Otherwise return to step 1.

## Code

### General Class `FinancialNetwork`

In [2]:
class FinancialNetwork(object):
    def __init__(self, D, C, C_hat, P, theta):
        #D, C, C_hat must be numpy.array object
        self.D = D
        self.C = C
        self.C_hat = C_hat
        self.P = P
        
        self.theta = theta
        
        n = self.C.shape[0]
        
        #Compute dependency matrix
        self.A = self.C_hat @ np.linalg.inv(np.eye(n) - self.C)
        
        #Compute the values of organizations
        self.v = self.A @ self.D@ self.P
        
        #Cost Function beta
        self.beta = self.P * 0.5
        
        #Set of failed organizations (1 -> fail)
        self._Z = np.zeros(n, dtype=int)
        
        #Vector b_tilde
        self.b_tilde = np.zeros(n)
        
        #Threshhold Value
        self.low_v = self.theta * self.v
        
        #Hierarchies of failures
        self._H =[]
        
    def main(self):
        Z_temp = copy.copy(self.Z)
        while True:
            H_temp = []
            X = self.A @ (self.D @ self.P - self.b_tilde) - self.low_v
            for i, tf in enumerate(X < 0):
                if tf:
                    Z_temp[i] = 1
                    self.b_tilde[i] = self.beta[i]
                    H_temp.append(i)
            self._H.append(H_temp)
            if (self._Z == Z_temp).all():
                break
            else:
                self.Z = Z_temp
                
        return self._H

The following `FinancialNetworkSpecial` class is specialized for the European Debt Cross-Holdings in the paper.

In [3]:
class FinancialNetworkSpecial(object):
    def __init__(self, A, P, P_2008, theta):
        self.A = A
        self.P = P
        self.P_2008 = P_2008
        
        self.theta = theta
        
        n = self.A.shape[0]
        
        #Threshhold Value
        self.low_v = self.theta * self.A @ self.P_2008
        
        #Cost Function beta
        self.beta = self.low_v * 0.5
        
        #Set of failed organizations (1 -> fail)
        self.Z = np.zeros(n, dtype=int)
        
        #Vector b_tilde
        self.b_tilde = np.zeros(n)
        
        #Hierarchies of failures
        self._H =[]
        
    def main(self):
        Z_temp = copy.copy(self.Z)
        while True:
            H_temp = []
            X = self.A @ (self.P - self.b_tilde) - self.low_v
            for i, tf in enumerate(X < 0):
                if tf:
                    Z_temp[i] = 1
                    self.b_tilde[i] = self.beta[i]
                    H_temp.append(i)
            self._H.append(H_temp)
            if (self.Z == Z_temp).all():
                break
            else:
                self.Z = Z_temp
        
        return self._H

- `A` : adjacency matrix
- `P` : Price vector in 2011
- ` P_2008` : Price vector in 2008, which is used for computing threshhold values

In [4]:
A = np.array([[0.71, 0.13, 0.13, 0.17, 0.07, 0.11], [0.18, 0.72, 0.12, 0.11, 0.09, 0.14], [0.00, 0.00, 0.67, 0.00, 0.00, 0.00], [0.07, 0.12, 0.03, 0.70, 0.03, 0.05], [0.01, 0.00, 0.02, 0.00, 0.67, 0.02], [0.03, 0.03, 0.02, 0.02, 0.14, 0.68]])
P = np.array([11.6, 14.9, 1.3, 9.2, 1.0, 6.3])
P_2008 = np.array([12.0, 15.4, 1.5, 9.8, 1.1, 6.7])

- 0 : France
- 1 : Germany
- 2 : Greece
- 3 : Italy
- 4 : Portugal
- 5 : Spain

In [5]:
FN = FinancialNetworkSpecial(A, P, P_2008, 0.935)
result = FN.main()
print(result)

[[2, 4], [2, 4, 5]]


This implies that Greece and Portugal fail first, and then Spain also fail with $\theta = 0.935$. All other countries do not fail.